In [20]:
%matplotlib inline
from pycbc.waveform import get_td_waveform
from pycbc.waveform import get_fd_waveform
from pycbc.filter import match, overlap
import pylab
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pycbc
import pycbc.psd
import h5py
import sys
from datetime import datetime

start = datetime.now()
print ("Start: %s" % start )

def GenTemplateTD (mass1, mass2, apx, eccentricity = .0, long_asc_nodes = .0, inclination = .0, f_low=30., sample_rate=4096):
    hptilde,hctilde = get_fd_waveform(approximant=apx,
                           mass1=mass1,
                           mass2=mass2,
                           eccentricity = eccentricity,
                           long_asc_nodes = long_asc_nodes,
                           inclination = inclination,
                           f_lower=f_low,
                           delta_f=1.0/4)
    
    # FFT it to the time-domain
    tlen = int(1.0 / (1.0/sample_rate) / hptilde.delta_f)
    hptilde.resize(tlen/2 + 1)
    hp = pycbc.types.TimeSeries(pycbc.types.zeros(tlen), delta_t=1.0/sample_rate)
    pycbc.fft.ifft(hptilde, hp)
    return hp

test_waveform = GenTemplateTD(1.1, 1.1, apx = "TaylorF2")

# Grab eccentric template bank
f_ecc = h5py.File('./ebank.hdf','r')
ecc_temp_mass1 = f_ecc['mass1'][:]
ecc_temp_mass2 = f_ecc['mass2'][:]
ecc_temp_eccentricity = f_ecc['eccentricity'][:]
ecc_temp_lan = f_ecc['long_asc_nodes'][:]
ecc_temp_inc = f_ecc['inclination'][:]

ecc_temp_length = len(ecc_temp_mass1)

#ecc_temp_apx = f_ecc['approximant'][:]
ecc_temp_apx = np.empty(len(ecc_temp_mass1), object)
for i_apx in range(0,len(ecc_temp_apx)):
    ecc_temp_apx[i_apx] = 'EccentricFD'

    




#test_ecc_wf = GenTemplateTD(1.1,1.10002, apx = "EccentricFD")
def GetMatch (template0,template1, psd_file = "H1L1-O1_C02_HARM_MEAN_PSD-1126051217-11203200.txt", f_low=30.):
    #resize two templates
    tlen = max(len(template0),len(template1))
    template1.resize(tlen)
    template0=template0.copy()
    template0.resize(tlen)
    
    #grab and use the psd file
    df = 1.0/template1.duration
    flen = tlen/2 +1
    my_psd = pycbc.psd.read.from_txt(filename = psd_file,
                                    length = flen,
                                    delta_f = df,
                                    low_freq_cutoff = f_low,
                                    is_asd_file = False)

    #calculate match
    m,i = match(template0,template1,psd=my_psd,low_frequency_cutoff=f_low)
    return m

    
waveform0_m1 = 1.59
waveform0_m2 = 1.59
waveform0_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(waveform0_m1, waveform0_m2) #compute mchirp of waveform0
local_matches = np.zeros(len(ecc_temp_mass1)) 
#iterate through the given template bank
for k in range(0,len(ecc_temp_mass1)):
    this_tp_m1 = ecc_temp_mass1[k]
    this_tp_m2 = ecc_temp_mass2[k]
    this_tp_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(this_tp_m1, this_tp_m2) #compute mchirp of one template
    diff_mchirp = abs(waveform0_mchirp-this_tp_mchirp)
    prozent_diff = diff_mchirp/waveform0_mchirp
    if (prozent_diff > 0.01):
        local_matches[k] = 0.
    else:
        # generate templates as needed
        one_relevant_temp = GenTemplateTD(mass1 = this_tp_m1, mass2 = this_tp_m2, apx = 'EccentricFD', 
                                                   eccentricity = ecc_temp_eccentricity[k], long_asc_nodes = ecc_temp_lan[k],
                                                   inclination = ecc_temp_inc[k])
        one_local_match= GetMatch(test_waveform, one_relevant_temp)
        local_matches[k] = one_local_match
        print(one_local_match)
    if k==0:
        print ("first one done")
    if k==100:
        print ("first 100 done")
    if k==200:
        print ("first 200 done")
    if k==500:
        print ("first 500 done")
    if k==1000:
        print ("first 1000 done")
    if k==2000:
        print ("first 2000 done")
    if k==5000:
        print ("first 5000 done")
    if k==8000:
        print ("first 8000 done")
    if k==10000:
        print ("first 10000 done")
    if k==100000:
        print ("first 100k done")
    if k==200000:
        print ("first 200k done")
    if k==300000:
        print ("first 300k done")
global_match = np.amax(local_matches)


this_glb_match = GetGlobalMatch(test_waveform, ecc_temps[0])
print ("this_glb_match: %s" % this_glb_match)
end = datetime.now()
print ("End: %s" % end)
duration = end - start
print ("runtime: %s" % duration)



Start: 2019-06-27 09:21:42.990984
0.09055874171512719
first one done
0.094745216883765
0.09044414027986174
0.08883438413200309
0.08600182149448764
0.09501101928910369


KeyboardInterrupt: 

In [ ]:
%matplotlib inline
from pycbc.waveform import get_fd_waveform
from pycbc.filter.matchedfilter import match
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pycbc
import pycbc.psd
import h5py
import sys

start = datetime.now()
print ("Start: %s" % start )

def GenTemplate (mass1, mass2, apx, eccentricity = .0, long_asc_nodes = .0, inclination = .0, f_low=30., freq_step=4):
    hptilde,hctilde = get_fd_waveform(approximant=apx,
                           mass1=mass1,
                           mass2=mass2,
                           eccentricity = eccentricity,
                           long_asc_nodes = long_asc_nodes,
                           inclination = inclination,
                           f_lower=f_low,
                           delta_f=1.0/freq_step)
    return hptilde

test_waveform = GenTemplate(1.1, 1.1, apx = "TaylorF2")

# Grab eccentric template bank
f_ecc = h5py.File('./ebank.hdf','r')
ecc_temp_mass1 = f_ecc['mass1'][:]
ecc_temp_mass2 = f_ecc['mass2'][:]
ecc_temp_eccentricity = f_ecc['eccentricity'][:]
ecc_temp_lan = f_ecc['long_asc_nodes'][:]
ecc_temp_inc = f_ecc['inclination'][:]

ecc_temp_length = len(ecc_temp_mass1)

#ecc_temp_apx = f_ecc['approximant'][:]
ecc_temp_apx = np.empty(len(ecc_temp_mass1), object)
for i_apx in range(0,len(ecc_temp_apx)):
    ecc_temp_apx[i_apx] = 'EccentricFD'

    

def GetMatch (template0,template1, psd_file = "H1L1-O1_C02_HARM_MEAN_PSD-1126051217-11203200.txt", f_low=30.):
    #resize two templates
    flen = max(len(template0),len(template1))
    template1.resize(flen)
    template0=template0.copy()
    template0.resize(flen)
    
    #grab and use the psd file
    df = 1.0/template1.duration
    my_psd = pycbc.psd.read.from_txt(filename = psd_file,
                                    length = flen,
                                    delta_f = df,
                                    low_freq_cutoff = f_low,
                                    is_asd_file = False)

    #calculate match
    m,i = match(template0,template1,psd=my_psd,low_frequency_cutoff=f_low)
    return m

    
waveform0_m1 = 1.1
waveform0_m2 = 1.1
waveform0_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(waveform0_m1, waveform0_m2) #compute mchirp of waveform0
local_matches = np.zeros(len(ecc_temp_mass1)) 
#iterate through the given template bank
rev_temp_counter = 0
for k in range(0,len(ecc_temp_mass1)):
    this_tp_m1 = ecc_temp_mass1[k]
    this_tp_m2 = ecc_temp_mass2[k]
    this_tp_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(this_tp_m1, this_tp_m2) #compute mchirp of one template
    diff_mchirp = abs(waveform0_mchirp-this_tp_mchirp)
    prozent_diff = diff_mchirp/waveform0_mchirp
    if (prozent_diff > 0.01):
        local_matches[k] = 0.
    else:
        # generate templates as needed
        one_relevant_temp = GenTemplate(mass1 = this_tp_m1, mass2 = this_tp_m2, apx = 'EccentricFD', 
                                                   eccentricity = ecc_temp_eccentricity[k], long_asc_nodes = ecc_temp_lan[k],
                                                   inclination = ecc_temp_inc[k])
        one_local_match= GetMatch(test_waveform, one_relevant_temp)
        local_matches[k] = one_local_match
        rev_temp_counter += 1
        print(one_local_match)
        if rev_temp_counter == 1:
            print ("First relevant counter found at k= %s " %k)
            print ("With mass1 = %s" % this_tp_m1)
            print ("With mass2 = %s" % this_tp_m2)
            print ("With mchirp = %s" % this_tp_mchirp)
            print ("With percentage mchirp difference = %s" % diff_mchirp)
            print ("With match = %s" % one_local_match)
    if k==0:
        print ("first one done")
    if k==100:
        print ("first 100 done")
    if k==200:
        print ("first 200 done")
    if k==500:
        print ("first 500 done")
    if k==1000:
        print ("first 1000 done")
    if k==2000:
        print ("first 2000 done")
    if k==5000:
        print ("first 5000 done")
    if k==8000:
        print ("first 8000 done")
    if k==10000:
        print ("first 10000 done")
    if k==100000:
        print ("first 100k done")
    if k==200000:
        print ("first 200k done")
    if k==300000:
        print ("first 300k done")
global_match = np.amax(local_matches)


this_glb_match = GetGlobalMatch(test_waveform, ecc_temps[0])
print ("this_glb_match: %s" % this_glb_match)
end = datetime.now()
print ("End: %s" % end)
duration = end - start
print ("runtime: %s" % duration)


Start: 2019-06-27 10:15:08.070899
first one done
first 100 done
first 200 done
first 500 done
first 1000 done
first 2000 done
first 5000 done
first 8000 done
first 10000 done
first 100k done
first 200k done
first 300k done
0.1588287403575779
First relevant counter found at k= 333571 
With mass1 = 1.1162549391562322
With mass2 = 1.1055591380748921
With mchirp = 0.9670873009024887
With percentage mchirp difference = 0.009481681276752085
With match = 0.1588287403575779
0.1622150146608058
0.17416111544896262
0.1507016890010899
0.17636066076825152
0.17688443587974728
0.15007751981238746
0.14418651890856019
0.1660803289445399
0.1626527373619494
0.1667582631818118
0.1607309096076443
0.1523779229203443
0.1523527510549445
0.1726322608626131
0.16578871637565512
0.15354541147096007
0.17024964124732533
0.16044474954857152
0.16510911238435746
0.15827186623762396
0.14449696801744594
0.17268965281426024
0.1725030325841628
0.16507483229260847
0.15069038687488376
0.15510484210677467
0.15546757291922766

0.2215229205261494
0.22080791040738795
0.18708144378926267
0.1986295784260336
0.17076475269422048
0.189030804724376
0.15967453596955264
0.19908020060673703
0.16701622611476333
0.15379530866529553
0.16790095947911574
0.18177378373826833
0.18857032009533664
0.18475525643035717
0.2702182558376096
0.1625948696349231
0.1706443954758003
0.1814221962644169
0.1708747620027392
0.1755476939563236
0.18556025845372243
0.1867863931167067
0.21627875332003627
0.16036494233393891
0.16479331975582012
0.19387367799403732
0.19151582021480928
0.1971507392666808
0.15942041986552316
0.16039822733186831
0.19728814928841376
0.19298452886855982
0.17572220379304646
0.1946794997903341
0.18131952662597237
0.2204008283620818
0.19776153183782824
0.18548452269024795
0.1616105679780545
0.16667659108629818
0.22831709784991394
0.16314825861860782
0.182534421709675
0.18350138453408585
0.1847160974006403
0.24184764524005647
0.15888128814420854
0.16438377457127465
0.21309586028107666
0.1892204443022174
0.23442817913915184

0.18405868127704972
0.2106864445265718
0.1679995790154232
0.1708973935571357
0.1599194642983883
0.15796506623517653
0.1812893098371462
0.17292162950339174
0.1765359654797371
0.15969281732474502
0.22253151319594425
0.1647573797148243
0.23122577031114463
0.16612454739625587
0.1981655820172647
0.17933150848979393
0.18221875304188137
0.2106249054103255
0.18260615940902628
0.18046234373048858
0.1784118269747179
0.155447462368597
0.16098096821315283
0.1757999815354546
0.1644418824401463
0.20015711589709917
0.19629965908532604
0.17891381155816816
0.197525471868921
0.19379111141642005
0.1705809501979772
0.15183534620879333
0.18120757923976172
0.2044480874159121
0.16163507738230687
0.1738531787652123
0.17223923482970113
0.19632103076257987
0.17671081649777945
0.15219705929032531
0.17305105410825283
0.15820717529208048
0.18386334566040455
0.20360239151020287
0.16978235930336066
0.1670097878829407
0.22384935464362085
0.22682068559130236
0.22634826564982075
0.21408558290061658
0.2102235606103212
0

0.21298067782383837
0.24248203371024388
0.22765474366038788
0.1578591749570732
0.15842754519151345
0.176009352996615
0.1880025934046961
0.1548554770837289
0.17870382122391554
0.17311957065964972
0.16619348406001805
0.18454520889635564
0.19214617731350955
0.1687870270915057
0.19535544654083564
0.16959363754992865
0.16124528134605706
0.18468312861215658
0.19806741417154028
0.17706895542817006
0.17584786496019292
0.17891840771141587
0.1626245120871018
0.21943195399501203
0.18713078268936523
0.187377337683022
0.15588554955968328
0.18095351201282048
0.16870182837239878
0.18438851342121812
0.18797726908948903
0.15935586329223655
0.17553911255767005
0.1611796776038444
0.15507967992956753
0.21017885949199114
0.17994530872761372
0.18226324663948348
0.15760001303086396
0.17045059279472727
0.17483649393630638
0.16236483429755624
0.18736960933326025
0.16625694097823054
0.16111996774301646
0.1840999173650918
0.1671099216767846
0.161172597267328
0.1586070131587319
0.15986047833263153
0.1603742785776

0.19356360220762045
0.1697196498146964
0.1714873929609624
0.17946234900747346
0.1673615176051898
0.1706465325726838
0.1762057631933024
0.21606167442843716
0.1845805488418534
0.1682642327606517
0.19261674990826438
0.2303083285783126
0.2015425243885198
0.22549639875815672
0.15070388181678587
0.20597052906018323
0.17113382936562493
0.18347214132518097
0.19752677446356837
0.1896416872732205
0.17696834481504198
0.1801553403104096
0.17266309896256965
0.18890916434049645
0.18966203360717002
0.1447505803503295
0.2179787373581685
0.1727606779036771
0.19452101660318818
0.17753549286579334
0.17828669313875542
0.20698029111636293
0.18247835176318233
0.15128746278756966
0.18883813139222302
0.15124133512963628
0.16687527568460434
0.16325630440715114
0.14926060281224743
0.1660174404028567
0.1911325684961876


In [ ]:
import pycbc

